# PHASM501 Project Stage One
## Tejan Shah
## Student Number: 14004521

The aim of this project is to develop a 2D finite element solver using Python and PyOpenCL.

Stage One of the project is to implement the basic functionality of the Grid objects that will be used to store and manipulate the finite element mesh data.

The element data comes in the form of a VTK file. As such, we begin by sourcing the PyOpenCl and VTK libraries necessary produce the Grid objects. 

This is done through Anaconda

In [2]:
!conda install -c conda-forge --yes pyopencl pocl
!conda install -y -c clinicalgraphics vtk

Solving environment: done

## Package Plan ##

  environment location: /home/nbuser/anaconda3_501

  added / updated specs: 
    - pocl
    - pyopencl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.3.33               |           py36_0         514 KB  conda-forge

The following packages will be DOWNGRADED:

    certifi: 2018.1.18-py36_0 --> 2017.11.5-py36_0 conda-forge
    conda:   4.4.9-py36_0     --> 4.3.33-py36_0    conda-forge


conda 4.3.33: ########################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Fetching package metadata .................
Solving package specifications: .

Package plan for installation in environment /home/nbuser/anaconda3_501:

The following packages will be UPDATED:

    conda: 4.3.33-py36_0 conda-forge --> 4.4.9-py36_0



We next import those libraries into our notebook for use later.

Here I am also importing the Numpy library for its mathematical manipulation tools and data structures, namely the Numpy Array.

In [23]:
import numpy as np
import pyopencl as cl
import vtk

I will also be making use of a Timer to test the relative speeds of the different solver implementations.

In [24]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start

Now the fun begins.
I will be making the Grid object in 3 different ways, with the first two being pure Python and the final using OpenCL.

To be able to understand this Grid object, we must first look at the final function defined within it.

'from_file' takes a string pointing to the location of the VTK file, and using the VTK library produces 2 numpy arrays. The first array corresponds to the list of vertices, while the second refers to the list of elements. You may notice that I am using an 'if' statement to ensure that my elements are of length 3. That is to say, I am ignoring any boundary elements in the VTK file that correspond to isolated lines or points, and only accepting triangular elements.

After this is complete, I instantiate the Grid object from within the 'from_file' function, as the constructor for the Grid object requires the vertices and elements arrays. This object is returned to be assigned to a variable outside of the class.

-------------------------------------------------------------------------------------------------------------------



Now we come to the constructor for the Grid object. The vertices and elements arrays are stored as member variables of the object. What follows is the calculation of the other Grid data we would like to store. As such, I wrap the next several lines of code in the Timer written above, so as to test the speed of this calculation.

Before continuing, it is worth a refresher on what exactly the vtk file stores, and what the arrays of vertices and elements represent.

Each vertex in the vertices array is a list of three points corresponding to the cartesian co-ordinates of that vertex. The vertex is not yet tied to any elements in particular, it is simply stored uniquely in the vertices array.

Each entry in the element array is a set of three numbers, where each number is an index of the vertices array. 

For a given element in the element array, the three numbers point to three distinct vertices in the vertices array; this set of three vertices represents the corners of the triangular element.


---

The first section of code is to work out each (3,3) set of corners for each element, and append it to a list of all Corners for all elements. This is stored in the Grid

Next we must find the Jacobian for a given element. The Jacobian can be thought of as the matrix representing the transformation of some reference element (which we choose for simplicity to be the element defined by the points (0,0), (1,0) and (0,1)) to the given element.

If we consider the points of our given element to be v0, v1 and v2, we can redefine our element as being made up of two edge vectors; both originating at v0 and pointing to v1 and v2 respecively. The Jacobian is then the transormation of the vector (0,0) -> (1,0) to v0 -> v1, and the vector (0,0) -> (0,1) to v0 -> v2. I hope this is clear, I'm not sure how to draw a picture in a python nutebook.

This redefinition allows us to construct the Jacobian from the points v0, v1 and v2, The first row of our Jacobian is v1 - v0, while the second row is v2 - v0.

Notice the Jacobian is (2,2) as we are dealing with 2D elements in a 2D (flat) space. I will attempt to generlise this later.

Next we find the Determinant and Inverse Transpose of out Jacobian using standard matrix methods.

The final method is somewhat intersting.

I am calculation the vector normal to the surface of the triangular element. As such it is sensible to use the cross product of the edge vectors we defined previously, and to simply normalise to give us a unit normal vector. However, it as worth recalling what exactly the cross product represents. It represents the area of the parallelogram produced by 2 vectors, pointing perpendicular to the plane of the two vectors. Also remember that the determinant of our Jacobian also represents this area. As such we can use the determinant as our normalisation factor (shown later).

In [25]:
class Grid(object):
    """A Grid class that can handle two dimensional flat and surface grids."""
    
    def __init__(self, vertices, elements, dtype='float64'):
        """
        Initialize a grid.
        
        Parameters
        ----------
        vertices: (N, 3) Array
            Vertices are stored in an (N, 3) array of floating point numbers.
        elements: (N, 3) Array
            Elements are stored in an (N, 3) array of type 'np.int32'.
        dtype: string
          dtype is either 'float64' or 'float32'. Internally, all structures and the
          vertices are converted to the format specified by 'dtype'. This is useful
          for OpenCL computations on GPUs that do not properly support double precision
          ('float64') types
        """
        #Store vertices and elements
        self._vertices=vertices
        self._elements=elements
        
        #Time computation
        with Timer() as t:
            #Create and store list of corners corresponding to each element
            Corners = []
            for i in range(len(self._elements)):
                Corners.append(self.corners(i))
            Corners=np.array(Corners)
            print("Corners:\n")
            print(Corners)
            self._Corners = Corners

            #Cconstruct Jacobians
            Jarray = []
            for c in Corners:
                J = np.zeros((2,2))
                #corners of element
                v0=c[0]
                v1=c[1]
                v2=c[2]
                
                #v1 - v0
                J[0][0]=v1[0]-v0[0]
                J[0][1]=v1[1]-v0[1]
                #v2 - v0
                J[1][0]=v2[0]-v0[0]
                J[1][1]=v2[1]-v0[1]
                Jarray.append(J)
            self._Jarray = np.array(Jarray)
            print("\nJacobian:\n")
            print(self._Jarray)


            #Calculate determinants
            detArray = []
            for j in self._Jarray:
                det = j[0][0]*j[1][1] - j[0][1]*j[1][0]
                detArray.append(det)
            self._det = np.array(detArray)
            print("\nDeterminant:\n")
            print(self._det)

            #Construct inverse transpose of the Jacobian
            
            #(a b) ->    1/    (d -c)
            #(c d)       det   (-b a)
            invT = []
            for i in range(len(self._Jarray)):
                j = self._Jarray[i]
                det = self._det[i]
                #Swap a and d
                j[0][0] = j[0][0] + j[1][1]
                j[1][1] = j[0][0] - j[1][1]
                j[0][0] = j[0][0] - j[1][1]

                #Swap b and c
                j[1][0] = j[1][0] + j[0][1]
                j[0][1] = j[1][0] - j[0][1]
                j[1][0] = j[1][0] - j[0][1]

                #flip signs of b and c
                j[1][0] = -1*(j[1][0])
                j[0][1] = -1*(j[0][1])

                j=(1/det)*j

                invT.append(j)
            self._invT = np.array(invT)
            print("\nInverse Transpose:\n")
            print(self._invT)

            Normal = []
            for j in self._Jarray:
                cross = np.cross(j[0],j[1])
                norm = np.linalg.norm(cross)
                n = cross/norm
                Normal.append(0)#set to 0 because I know the normal only has a z component, will generlise in future
                Normal.append(0)
                Normal.append(n)
            self._Normal=np.array(Normal).reshape(len(self._elements),3)
            print("\nNormal:\n")
            print(self._Normal)
        #Time taken to calculate
        print("Time: ",t.interval)
        
    #return Grid properties
    @property
    def vertices(self):
        """Return the vertices."""
        return self._vertices
        
    @property
    def elements(self):
        """Return the elemenets."""
        return self._elements
    
    def corners(self, element_index):
        """Return the corners of a given element as (3, 3) array"""
        e = self._elements[element_index]
        corner = []
        for i in range(len(e)):
            corner.append(self._vertices[e[i]])
        return np.array(corner)

    
    def integration_element(self, element_index):
        """Return |det J| for a given element."""
        return self._det[element_index]
    
    def normal(self, element_index):
        """Return the exterior pointing normal of an element."""
        return self._Normal[element_index]
    
    def inverse_jacobian_transpose(self, element_index):
        """Return the (3, 3) inverse Jacobian transpose of an element."""
        return self._invT[element_index]
    
    @classmethod
    def from_file(cls, file_name):
        """Read a mesh from a vtk file."""
        #prepare the reader to read the vtk file
        reader = vtk.vtkUnstructuredGridReader()
        reader.SetFileName(file_name)
        reader.Update()
        
        #get the data from the reader (i.e. from the vtk)
        data = reader.GetOutput()
    
        #produce a list of vertices
        points = []
        for i in range(data.GetNumberOfPoints()):
            points.append(list(data.GetPoint(i)))
        
        points = np.array(points)
        
        #produce a list of triangular elements
        cells =[]
        for i in range(data.GetNumberOfCells()):
            cell = []
            if(data.GetCell(i).GetNumberOfPoints() != 3):
                continue
            cell.append(data.GetCell(i).GetPointId(0))
            cell.append(data.GetCell(i).GetPointId(1))
            cell.append(data.GetCell(i).GetPointId(2))
            cells.append(cell)
        cells=np.array(cells)
        
        #instantiate the Grid object
        grid = Grid(points,cells)
        return grid

In [26]:
grid = Grid.from_file('lshape.vtk')

Corners:

[[[ 1.       1.       0.     ]
  [ 0.96875  0.96875  0.     ]
  [ 1.       0.9375   0.     ]]

 [[ 0.96875  0.96875  0.     ]
  [ 0.96875  0.90625  0.     ]
  [ 1.       0.9375   0.     ]]

 [[ 0.96875  0.96875  0.     ]
  [ 0.9375   0.9375   0.     ]
  [ 0.96875  0.90625  0.     ]]

 ...

 [[ 0.5625  -0.5      0.     ]
  [ 0.53125 -0.46875  0.     ]
  [ 0.53125 -0.53125  0.     ]]

 [[ 0.5625  -0.5      0.     ]
  [ 0.5625  -0.4375   0.     ]
  [ 0.53125 -0.46875  0.     ]]

 [[ 0.53125 -0.53125  0.     ]
  [ 0.53125 -0.46875  0.     ]
  [ 0.5     -0.5      0.     ]]]

Jacobian:

[[[-0.03125 -0.03125]
  [ 0.      -0.0625 ]]

 [[ 0.      -0.0625 ]
  [ 0.03125 -0.03125]]

 [[-0.03125 -0.03125]
  [ 0.      -0.0625 ]]

 ...

 [[-0.03125  0.03125]
  [-0.03125 -0.03125]]

 [[ 0.       0.0625 ]
  [-0.03125  0.03125]]

 [[ 0.       0.0625 ]
  [-0.03125  0.03125]]]

Determinant:

[0.00195313 0.00195313 0.00195313 ... 0.00195312 0.00195312 0.00195312]

Inverse Transpose:

[[[-32.  -0.

Next, I have rewritten the python code above but with more of an OpenCl style. As OpenCl can be used to effectively remove loops from programs, I wanted to write the above python code in a single 'for loop'. This would then allow me to transfer to OpenCl and eliminate all loops. This was achieved by flattening all arrays into 1D and traversing them using a single global ID. 

The importing from the VTk is the same across all versions of Grid, so I will skip over that.

I begin by creating empty arrays to store the output, much like the OpenCL output buffers. Then with each iteration, I reference the necessary entry in my input arrays by indexing using this global ID. The order of computation is the same as before, first corners then jacobians etc. Notice the global ID is often multiplied by some factor. This factor depends on the size of a given row within the array e.g. the flattened corners array has 9 entries per element, so the multiplication factor is 9.

In [27]:
class Grid(object):
    """A Grid class that can handle two dimensional flat and surface grids."""
    
    def __init__(self, vertices, elements, dtype='float64'):
        """
        Initialize a grid.
        
        Parameters
        ----------
        vertices: (N, 3) Array
            Vertices are stored in an (N, 3) array of floating point numbers.
        elements: (N, 3) Array
            Elements are stored in an (N, 3) array of type 'np.int32'.
        dtype: string
          dtype is either 'float64' or 'float32'. Internally, all structures and the
          vertices are converted to the format specified by 'dtype'. This is useful
          for OpenCL computations on GPUs that do not properly support double precision
          ('float64') types
        """
        self._vertices=vertices
        self._elements=elements
        
        
        c = [None]*len(self._elements)*9
        J = [None]*len(self._elements)*4
        Det = [None]*len(self._elements)
        InvT = [None]*len(J)
        Norm = [None]*len(self._elements)*3
        
        with Timer() as t:
            for gid in range(len(elements)):
                v = self._vertices.flatten()
                e = self._elements.flatten()

                #vij = v[e[(gid+i)*3]*3 + j]

                c[gid*9] = v[e[gid*3]*3] #v00
                c[gid*9+1] = v[e[(gid*3)]*3 +1] #v01
                c[gid*9+2] = v[e[(gid*3)]*3 +2] #v02
                c[gid*9+3] = v[e[(gid*3+1)]*3] #v10
                c[gid*9+4] = v[e[(gid*3+1)]*3 + 1] #v11
                c[gid*9+5] = v[e[(gid*3+1)]*3 + 2] #v12
                c[gid*9+6]= v[e[(gid*3+2)]*3 ] #v20
                c[gid*9+7] = v[e[(gid*3+2)]*3 + 1] #v21
                c[gid*9+8] = v[e[(gid*3+2)]*3 + 2] #v22

                J[gid*4] = c[gid*9+3] - c[gid*9]
                J[gid*4+1] = c[gid*9+4] - c[gid*9+1]
                J[gid*4+2] = c[gid*9+6] - c[gid*9]
                J[gid*4+3] = c[gid*9+7] - c[gid*9+1]

                Det[gid] = J[gid*4]*J[gid*4+3] - J[gid*4+1]*J[gid*4+2]

                det = (1/Det[gid])
                InvT[gid*4] = det*J[gid*4+3]
                InvT[gid*4+1] = -1*det*J[gid*4+2]
                InvT[gid*4+2] = -1*det*J[gid*4+1]
                InvT[gid*4+3] = det*J[gid*4]
                
                Norm[gid*3] = 0
                Norm[gid*3+1] = 0
                Norm[gid*3+2] = Det[gid]/(np.sqrt(Det[gid]*Det[gid]))
                
            print("Corners:\n")
            print(np.array(c).reshape(len(elements),3,3))
            print("\nJacobian:\n")
            print(np.array(J).reshape(len(elements),2,2))
            print("\nDeterminant:\n")
            print(np.array(Det))
            print("\nInverse Transpose:\n")
            print(np.array(InvT).reshape(len(self._elements),2,2))
            print("\nNormal:\n")
            print(np.array(Norm).reshape(len(self._elements),3))
        
        print(t.interval)
            
    @property
    def vertices(self):
        """Return the vertices."""
        return self._vertices
        
    @property
    def elements(self):
        """Return the elemenets."""
        return self._elements
    
    def corners(self, element_index):
        """Return the corners of a given element as (3, 3) array"""
        e = self._elements[element_index]
        corner = []
        for i in range(len(e)):
            corner.append(self._vertices[e[i]])
        return np.array(corner)
    
    #calc jacobian 4 buffers v0, v1, v2 and output
    
    def integration_element(self, element_index):
        """Return |det J| for a given element."""
        #2 buffers jacobian and output
        pass
    
    def normal(self, element_index):
        """Return the exterior pointing normal of an element."""
        #3 buffers x, y, output of 2 buffers jacobian and output
        pass
    
    def inverse_jacobian_transpose(self, element_index):
        """Return the (3, 3) inverse Jacobian transpose of an element."""
        #3 buffers jacobian, det, output
        pass
    
    @classmethod
    def from_file(cls, file_name):
        """Read a mesh from a vtk file."""
        reader = vtk.vtkUnstructuredGridReader()
        reader.SetFileName(file_name)
        reader.Update()
        
        data = reader.GetOutput()
    
        points = []
        for i in range(data.GetNumberOfPoints()):
            points.append(list(data.GetPoint(i)))
        
        points = np.array(points)

        cells =[]
        for i in range(data.GetNumberOfCells()):
            cell = []
            if(data.GetCell(i).GetNumberOfPoints() != 3):
                continue
            cell.append(data.GetCell(i).GetPointId(0))
            cell.append(data.GetCell(i).GetPointId(1))
            cell.append(data.GetCell(i).GetPointId(2))
            cells.append(cell)
        cells=np.array(cells)
        
        grid = Grid(points,cells)
        return grid


In [28]:
grid = Grid.from_file('lshape.vtk')

Corners:

[[[ 1.       1.       0.     ]
  [ 0.96875  0.96875  0.     ]
  [ 1.       0.9375   0.     ]]

 [[ 0.96875  0.96875  0.     ]
  [ 0.96875  0.90625  0.     ]
  [ 1.       0.9375   0.     ]]

 [[ 0.96875  0.96875  0.     ]
  [ 0.9375   0.9375   0.     ]
  [ 0.96875  0.90625  0.     ]]

 ...

 [[ 0.5625  -0.5      0.     ]
  [ 0.53125 -0.46875  0.     ]
  [ 0.53125 -0.53125  0.     ]]

 [[ 0.5625  -0.5      0.     ]
  [ 0.5625  -0.4375   0.     ]
  [ 0.53125 -0.46875  0.     ]]

 [[ 0.53125 -0.53125  0.     ]
  [ 0.53125 -0.46875  0.     ]
  [ 0.5     -0.5      0.     ]]]

Jacobian:

[[[-0.03125 -0.03125]
  [ 0.      -0.0625 ]]

 [[ 0.      -0.0625 ]
  [ 0.03125 -0.03125]]

 [[-0.03125 -0.03125]
  [ 0.      -0.0625 ]]

 ...

 [[-0.03125  0.03125]
  [-0.03125 -0.03125]]

 [[ 0.       0.0625 ]
  [-0.03125  0.03125]]

 [[ 0.       0.0625 ]
  [-0.03125  0.03125]]]

Determinant:

[0.00195313 0.00195313 0.00195313 ... 0.00195312 0.00195312 0.00195312]

Inverse Transpose:

[[[-32.  -0.

Finally the main star of the show.

After imporitng from the VTK file as before, I create the OpenCl context, queue, input buffers, output arrays and corresponding buffers, and finally the kernel. The kernel mirrors the python shown above.

A few noteworthy differences:

1) There is an extra factor of 2 in the elements buffer indexing. This is because each interger entry is apparently padded with zeros. This may be a quirk of OpenCl or due to me passing in a numpy array of integers. This issue does not occur with float buffers. 

2) As I calculate the (3,3) corners, I decided to caluculate the (3,3) Jacobians, since the data was there to be used. Since the given example data is in the plane, I am comfortable in simply filling in the final row of the Jacobian, but I suspect the way in which the Jacobian is calculated may differ. The difficulty lies in the fact that the Jacobian must be NxN, so having a 2x3 matrix (from our vector calculation method v1 - v0 etc) is unacceptable.

3) I am still calculating the determinant based off of the 2x2 Jacobians, as in 3D our determinants would be 0 (the elements are 2D so have 0 volume so would give 0 determinants)

4) the Inverse transpose is still 2x2 as a 3x3 Jacobian with all zeros in the last row would be singular. Again I hope to generlise in future to allow for elements not in the plane, or for 3D elements.

5) the Normals still have only 0 for the x and y entries, as we know the grid is in the plane. Will generlise in future.

6) The dtype is set to float32 as default as that is what is required to run on Azure notebooks. The program will run with float64 given the appropriate hardware.

In [29]:
class Grid(object):
    """A Grid class that can handle two dimensional flat and surface grids."""
    
    def __init__(self, vertices, elements, dtype='float32'):
        """
        Initialize a grid.
        
        Parameters
        ----------
        vertices: (N, 3) Array
            Vertices are stored in an (N, 3) array of floating point numbers.
        elements: (N, 3) Array
            Elements are stored in an (N, 3) array of type 'np.int32'.
        dtype: string
          dtype is either 'float64' or 'float32'. Internally, all structures and the
          vertices are converted to the format specified by 'dtype'. This is useful
          for OpenCL computations on GPUs that do not properly support double precision
          ('float64') types
        """
        self._vertices=vertices
        self._elements=elements
        
        e_np = self._elements
        v_np = self._vertices.astype(dtype)
        
        #make OpenCl context
        ctx = cl.create_some_context()
        queue = cl.CommandQueue(ctx)
        mf = cl.mem_flags
        
        #create input buffers
        e_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=e_np)
        v_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=v_np)
        
        #create output arrays
        c_np = np.empty([len(elements),3,3]).astype(dtype)
        J_np = np.empty([len(elements),3,3]).astype(dtype)
        D_np = np.empty([len(elements)]).astype(dtype)
        I_np = np.empty([len(elements),2,2]).astype(dtype)
        N_np = np.empty([len(elements),3]).astype(dtype)
        
        #make output buffers based on output arrays
        c_g = cl.Buffer(ctx, mf.WRITE_ONLY,c_np.nbytes)
        J_g = cl.Buffer(ctx, mf.WRITE_ONLY,J_np.nbytes)
        D_g = cl.Buffer(ctx, mf.WRITE_ONLY,D_np.nbytes)
        I_g = cl.Buffer(ctx, mf.WRITE_ONLY,I_np.nbytes)
        N_g = cl.Buffer(ctx, mf.WRITE_ONLY,N_np.nbytes)

        kernel = """
            __kernel void FiniteElement(const __global int* e_g,
                                        const __global float* v_g,
                                        __global float* c_g,
                                        __global float* J_g,
                                        __global float* Det,
                                        __global float* InvT,
                                        __global float* N_g){

            const int gid = get_global_id(0);
            
            int e1 = e_g[2*(3*gid)];
            int e2 = e_g[2*(3*gid+1)];
            int e3 = e_g[2*(3*gid+2)];

            c_g[gid*9] = v_g[e1*3];
            c_g[gid*9+1] = v_g[e1*3 +1];
            c_g[gid*9+2] = v_g[e1*3 +2];
            c_g[gid*9+3] = v_g[e2*3];
            c_g[gid*9+4] = v_g[e2*3 +1]; 
            c_g[gid*9+5] = v_g[e2*3 +2]; 
            c_g[gid*9+6] = v_g[e3*3]; 
            c_g[gid*9+7]= v_g[e3*3 + 1]; 
            c_g[gid*9+8] = v_g[e3*3 + 2]; 
            
            J_g[gid*9] = c_g[gid*9+3] - c_g[gid*9];
            J_g[gid*9+1] = c_g[gid*9+4] - c_g[gid*9+1];
            J_g[gid*9+2] = c_g[gid*9+5] - c_g[gid*9+2];
            
            J_g[gid*9+3] = c_g[gid*9+6] - c_g[gid*9];
            J_g[gid*9+4] = c_g[gid*9+7] - c_g[gid*9+1];
            J_g[gid*9+5] = c_g[gid*9+8] - c_g[gid*9+2];
            
            J_g[gid*9+6] = 0;
            J_g[gid*9+7] = 0;
            J_g[gid*9+8] = 0;
            
            Det[gid] = J_g[gid*9]*J_g[gid*9+4] - J_g[gid*9+1]*J_g[gid*9+3];
            
            
            float invdet = (1/Det[gid]);
            InvT[gid*4] = invdet*J_g[gid*9+3];
            InvT[gid*4+1] = -1*invdet*J_g[gid*9+2];
            InvT[gid*4+2] = -1*invdet*J_g[gid*9+1];
            InvT[gid*4+3] = invdet*J_g[gid*9];
            
            float absdet = (Det[gid]*Det[gid])/Det[gid];
            N_g[gid*3] = 0;
            N_g[gid*3+1] = 0;
            N_g[gid*3+2] = Det[gid]/absdet;
            
            }
            """
        #build kernel
        prg = cl.Program(ctx,kernel).build()
        FE = prg.FiniteElement
        #run and time program
        with Timer() as t:
            FE(queue, c_np.shape,None,e_g,v_g,c_g,J_g,D_g,I_g,N_g)
            #copy results from output buffers to appropriate arrays
            cl.enqueue_copy(queue,c_np,c_g)
            self._corners = c_np
            cl.enqueue_copy(queue,J_np,J_g)
            self._jacobian = J_np
            cl.enqueue_copy(queue,D_np,D_g)
            self._determinant = D_np
            cl.enqueue_copy(queue,I_np,I_g)
            self._invT = I_np
            cl.enqueue_copy(queue,N_np,N_g)
            self._normal = N_np
        
            print("Corners:\n\n",c_np,"\n\n Jacobians:\n\n",J_np,
                  "\n\nDeterminants:\n\n",D_np,"\n\n Inverse Transposes:\n\n",I_np
                 ,"\n\nNormal:\n\n",N_np)
        print("Time: ",t.interval)
    
    @property
    def vertices(self):
        """Return the vertices."""
        return self._vertices
        
    @property
    def elements(self):
        """Return the elemenets."""
        return self._elements
    
    def corners(self, element_index):
        """Return the corners of a given element as (3, 3) array"""
        return self._corners[element_index]
    
    
    def integration_element(self, element_index):
        """Return |det J| for a given element."""

        return self._determinant[element_index]
    
    def normal(self, element_index):
        """Return the exterior pointing normal of an element."""
        return self._normal
    
    def inverse_jacobian_transpose(self, element_index):
        """Return the (3, 3) inverse Jacobian transpose of an element."""
        return self._invT[element_index]
    
    
    @classmethod
    def from_file(cls, file_name):
        """Read a mesh from a vtk file."""
        reader = vtk.vtkUnstructuredGridReader()
        reader.SetFileName(file_name)
        reader.Update()
        
        data = reader.GetOutput()
    
        points = []
        for i in range(data.GetNumberOfPoints()):
            points.append(list(data.GetPoint(i)))
        
        points = np.array(points)

        cells =[]
        for i in range(data.GetNumberOfCells()):
            cell = []
            if(data.GetCell(i).GetNumberOfPoints() != 3):
                continue
            cell.append(data.GetCell(i).GetPointId(0))
            cell.append(data.GetCell(i).GetPointId(1))
            cell.append(data.GetCell(i).GetPointId(2))
            cells.append(cell)
        cells=np.array(cells)
        
        grid = Grid(points,cells)
        return grid

In [30]:
grid = Grid.from_file('lshape.vtk')

Corners:

 [[[ 1.       1.       0.     ]
  [ 0.96875  0.96875  0.     ]
  [ 1.       0.9375   0.     ]]

 [[ 0.96875  0.96875  0.     ]
  [ 0.96875  0.90625  0.     ]
  [ 1.       0.9375   0.     ]]

 [[ 0.96875  0.96875  0.     ]
  [ 0.9375   0.9375   0.     ]
  [ 0.96875  0.90625  0.     ]]

 ...

 [[ 0.5625  -0.5      0.     ]
  [ 0.53125 -0.46875  0.     ]
  [ 0.53125 -0.53125  0.     ]]

 [[ 0.5625  -0.5      0.     ]
  [ 0.5625  -0.4375   0.     ]
  [ 0.53125 -0.46875  0.     ]]

 [[ 0.53125 -0.53125  0.     ]
  [ 0.53125 -0.46875  0.     ]
  [ 0.5     -0.5      0.     ]]] 

 Jacobians:

 [[[-0.03125 -0.03125  0.     ]
  [ 0.      -0.0625   0.     ]
  [ 0.       0.       0.     ]]

 [[ 0.      -0.0625   0.     ]
  [ 0.03125 -0.03125  0.     ]
  [ 0.       0.       0.     ]]

 [[-0.03125 -0.03125  0.     ]
  [ 0.      -0.0625   0.     ]
  [ 0.       0.       0.     ]]

 ...

 [[-0.03125  0.03125  0.     ]
  [-0.03125 -0.03125  0.     ]
  [ 0.       0.       0.     ]]

 [[ 0.     

As you can see from the outputs and timings of the three methods, while the outputs agree the OpenCl is significantly faster. The next step would be to continue generlising for different mesh grids, and to allow for specifically GPU computing (not something I wanted to include here as it cannot be tested on the Azure notebook).